# ShohnomaLLM - Обучение модели

Fine-tuning **Qwen3-4B** для генерации персидской поэзии (арабица).

**Настройки:**
- ✅ Обучаем на **оригинальной арабице** (не транслитерация!)
- ✅ 30K лучших стихов (отфильтровано из 119K)
- ✅ 1 эпоха (достаточно для 30K)

**Время: ~15-20 минут на A100**

In [ ]:
# Проверка GPU
!nvidia-smi

In [ ]:
# Установка зависимостей
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# Клонируем репозиторий ShohnomaLLM
!git clone https://github.com/Kuchizu/ShohnomaLLM.git
%cd ShohnomaLLM
!pip install -r requirements.txt -q

In [ ]:
# Подключение Google Drive (для сохранения модели)
from google.colab import drive
drive.mount('/content/drive')

# Пути
REPO_DIR = "/content/ShohnomaLLM"
DATA_DIR = f"{REPO_DIR}/data"  # Данные из репозитория
MODEL_DIR = "/content/drive/MyDrive/ShohnomaLLM/models"  # Модели в Drive

# Создаём директории
!mkdir -p {DATA_DIR}/processed {DATA_DIR}/training {MODEL_DIR}

## 1. Подготовка данных

Используем модули из репозитория

In [ ]:
import sys
sys.path.insert(0, REPO_DIR)

from training.format_dataset import DatasetFormatter
from training.config import get_config

# Загружаем конфигурацию для A100
config = get_config("colab_a100")
print(f"Модель: {config.model.base_model}")
print(f"LoRA rank: {config.lora.r}")
print(f"Batch size: {config.training.per_device_train_batch_size}")
print(f"Gradient accumulation: {config.training.gradient_accumulation_steps}")

In [ ]:
# Загрузка данных с HuggingFace (БЕЗ транслитерации - используем оригинал!)
import os
import json
from tqdm import tqdm

hf_data = f"{DATA_DIR}/raw/ganjoor_hf/all_poems.jsonl"

# Если данных нет - скачиваем
if not os.path.exists(hf_data):
    print("Загрузка датасета с HuggingFace (119K стихов)...")
    from datasets import load_dataset
    
    os.makedirs(f"{DATA_DIR}/raw/ganjoor_hf", exist_ok=True)
    
    dataset_hf = load_dataset("mabidan/ganjoor", split="train")
    print(f"Загружено: {len(dataset_hf)} стихов")
    
    # Сохраняем БЕЗ транслитерации - оригинальный персидский!
    print("Сохранение оригинальных текстов (арабица)...")
    with open(hf_data, 'w', encoding='utf-8') as f:
        for item in tqdm(dataset_hf, desc="Обработка"):
            text = item.get("text", "")
            if not text or len(text) < 30:
                continue
            
            poem_data = {
                "id": f"hf_{item.get('id', 0)}",
                "poet": item.get("poet", ""),
                "text_persian": text,  # Оригинал!
                "form": "other",
                "source": "huggingface",
            }
            json.dump(poem_data, f, ensure_ascii=False)
            f.write('\n')
    
    print(f"Сохранено: {hf_data}")

# Подготовка датасета
from training.format_dataset import prepare_full_dataset

stats = prepare_full_dataset(
    raw_dir=f"{DATA_DIR}/raw",
    processed_dir=f"{DATA_DIR}/processed",
    training_dir=f"{DATA_DIR}/training",
)
print(f"\nВсего подготовлено: {stats.get('total', 0)} примеров")

## 2. Загрузка модели

In [ ]:
from unsloth import FastLanguageModel
import torch

# Загрузка модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config.model.base_model,
    max_seq_length=config.model.max_seq_length,
    dtype=None,
    load_in_4bit=config.model.load_in_4bit,
)

print(f"Модель загружена: {config.model.base_model}")

In [ ]:
# Добавляем LoRA (БЕЗ gradient checkpointing для скорости!)
model = FastLanguageModel.get_peft_model(
    model,
    r=config.lora.r,
    target_modules=config.lora.target_modules,
    lora_alpha=config.lora.lora_alpha,
    lora_dropout=config.lora.lora_dropout,
    bias=config.lora.bias,
    use_gradient_checkpointing=False,  # ВАЖНО: выключено для скорости на A100!
    random_state=42,
)

print("LoRA добавлен (без gradient checkpointing)")
model.print_trainable_parameters()

## 3. Обучение

In [ ]:
from datasets import load_dataset

# Лимит данных для быстрого обучения
MAX_SAMPLES = 30000

# Загрузка датасета
dataset = load_dataset(
    'json',
    data_files={
        'train': f"{DATA_DIR}/training/train.jsonl",
        'validation': f"{DATA_DIR}/training/val.jsonl",
    }
)

print(f"Загружено - Train: {len(dataset['train'])}, Val: {len(dataset['validation'])}")

# Ограничиваем размер для скорости
if len(dataset['train']) > MAX_SAMPLES:
    dataset['train'] = dataset['train'].shuffle(seed=42).select(range(MAX_SAMPLES))
    dataset['validation'] = dataset['validation'].shuffle(seed=42).select(range(min(3000, len(dataset['validation']))))
    print(f"Ограничено до {MAX_SAMPLES} примеров для обучения")

print(f"Итого - Train: {len(dataset['train'])}, Val: {len(dataset['validation'])}")

In [ ]:
# Настройка параметров - ОПТИМИЗИРОВАНО ДЛЯ A100 + 30K данных
from trl import SFTTrainer, SFTConfig

dataset_size = len(dataset['train'])
print(f"Размер датасета: {dataset_size}")

# 1 эпоха для 30K данных
num_epochs = 1
total_steps = dataset_size // 32  # batch=8 x accum=4 = 32
eval_steps = max(100, total_steps // 5)
save_steps = eval_steps

print(f"Epochs: {num_epochs}")
print(f"Ожидаемых шагов: ~{total_steps}")
print(f"Ожидаемое время: ~{total_steps // 60 + 5}-{total_steps // 45 + 5} минут")

# Trainer с PACKING
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    
    args=SFTConfig(
        output_dir="./outputs",
        dataset_text_field="text",
        
        # === PACKING ===
        packing=True,
        max_seq_length=512,  # Уменьшено для экономии памяти
        dataset_num_proc=4,
        
        # === BATCH SIZE для A100 (оптимизировано) ===
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=4,  # Эффективный batch = 32
        
        # === Learning Rate ===
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,
        num_train_epochs=num_epochs,
        
        # === Precision ===
        bf16=True,
        optim="adamw_8bit",
        weight_decay=0.01,
        max_grad_norm=1.0,
        
        # === Dataloader ===
        dataloader_num_workers=4,
        dataloader_pin_memory=True,
        
        # === Logging ===
        logging_steps=25,
        eval_steps=eval_steps,
        eval_strategy="steps",
        save_steps=save_steps,
        save_strategy="steps",
        save_total_limit=2,
        load_best_model_at_end=True,
        
        seed=42,
        report_to="none",
    ),
)

In [ ]:
# Запуск обучения
print("Начало обучения...")
trainer.train()

## 4. Сохранение модели

In [ ]:
# Сохраняем LoRA
lora_path = f"{MODEL_DIR}/tajik-poetry-lora"
model.save_pretrained(lora_path)
tokenizer.save_pretrained(lora_path)
print(f"LoRA сохранён: {lora_path}")

In [ ]:
# Объединяем LoRA с базовой моделью (16-bit)
merged_path = f"{MODEL_DIR}/tajik-poetry-4b"

model.save_pretrained_merged(
    merged_path,
    tokenizer,
    save_method="merged_16bit",
)
print(f"Merged модель: {merged_path}")

## 5. Тестирование

In [ ]:
# Режим inference
FastLanguageModel.for_inference(model)

# Системный промпт
from training.format_dataset import SYSTEM_PROMPT

def generate_poem(prompt, max_tokens=256, temperature=0.8):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True,
    )
    
    return tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True,
    )

In [ ]:
# Тестовые промпты (персидский)
test_prompts = [
    "رباعی بنویس",           # Напиши рубаи
    "غزل عاشقانه بساز",      # Создай любовную газель
    "شعر درباره بهار بنویس",  # Напиши стих о весне
]

for prompt in test_prompts:
    print(f"\n{'='*50}")
    print(f"Запрос: {prompt}")
    print(f"{'='*50}")
    print(generate_poem(prompt))

## 6. Экспорт в GGUF (опционально)

Для запуска на CPU через llama.cpp

In [ ]:
# Экспорт в GGUF
gguf_path = f"{MODEL_DIR}/tajik-poetry-q4"

model.save_pretrained_gguf(
    gguf_path,
    tokenizer,
    quantization_method="q4_k_m",
)
print(f"GGUF сохранён: {gguf_path}")

In [ ]:
print("\n" + "="*50)
print("Обучение завершено!")
print("="*50)
print(f"\nМодели сохранены в Google Drive: {MODEL_DIR}")
print("\nСкачайте модель и используйте локально:")
print("  python -m cli.generate --model путь/к/модели")